# **FOUNDAMENTAL APPROACH - COCHRANE (1992) VAR MODEL EXTENSION**

### **1) REQUIREMENTS SETUP**

In [90]:
# !pip install -r requirements.txt

In [91]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np

### **2) MONTHLY DATA ANALYSIS**

##### **2.1) VARIABLES CONSTRUCTION**

In [92]:
# Log Risk Free Rate
rf_df = pd.read_csv("../data_extraction/raw_df/risk_free_monthly_df.csv")
rf_df = rf_df.rename(columns={"1-month Yield - US Treasury Securities":"Rf_t"})
rf_df["Date"] = pd.to_datetime(rf_df["Date"])

# Data Cleaning for NaN, -inf, 0.0 
rf_df["Rf_t"] = pd.to_numeric(rf_df["Rf_t"], errors="coerce") 
rf_df["Rf_t"] = rf_df["Rf_t"].replace([0, 0.0, -np.inf, np.inf], np.nan)

# (!!!) Either forward or backward filling
rf_df["Rf_t"] = rf_df["Rf_t"].ffill().bfill()

rf_df["rf_t"] = np.log(rf_df["Rf_t"])

rf_df.tail()

,Date,Rf_t,rf_t
282,2025-07-01,4.32,1.463255
283,2025-08-01,4.49,1.501853
284,2025-09-01,4.41,1.483875
285,2025-10-01,4.17,1.427916
286,2025-11-01,4.06,1.401183


In [93]:
# Log Synthetic Index Price
p_df = pd.read_csv("../data_extraction/raw_df/synthetic_price_monthly_df.csv")
p_df["p_t"] = np.log(p_df["Synthetic Index Close Price"])
p_df = p_df.rename(columns={"Synthetic Index Close Price":"P_t"})
p_df["Date"] = pd.to_datetime(p_df["Date"].astype(str).str[:10], format="%Y-%m-%d")
p_df.tail()

,Date,P_t,p_t
282,2025-07-01,333.590679,5.809915
283,2025-08-01,340.899950,5.831589
284,2025-09-01,348.953859,5.854940
285,2025-10-01,366.698338,5.904540
286,2025-11-01,375.264182,5.927630


In [94]:
# Log Synthetic Index Dividend
d_df = pd.read_csv("../data_extraction/raw_df/synthetic_div_monthly_df.csv")
d_df["d_t"] = np.log(d_df["Synthetic Index Dividend"])
d_df = d_df.rename(columns={"Synthetic Index Dividend": "D_t"})
d_df["Date"] = pd.to_datetime(d_df["Date"].astype(str).str[:10], format="%Y-%m-%d")
d_df.tail()

,Date,D_t,d_t
282,2025-07-01,0.055846,-2.885165
283,2025-08-01,0.253449,-1.372591
284,2025-09-01,0.216313,-1.531027
285,2025-10-01,0.076930,-2.564857
286,2025-11-01,0.061543,-2.788022


In [95]:
# Merged Variable Dataset
monthly_df = pd.merge(p_df, d_df, on="Date", how="outer")
monthly_df = pd.merge(monthly_df, rf_df, on="Date", how="outer")

monthly_df

,Date,P_t,p_t,D_t,d_t,Rf_t,rf_t
0,2002-01-01,6.747791,1.909215,0.000382,-7.869682,1.69,0.524729
1,2002-02-01,6.713567,1.904130,0.006262,-5.073240,1.69,0.524729
2,2002-03-01,6.619971,1.890091,0.002720,-5.907281,1.78,0.576613
3,2002-04-01,6.720307,1.905134,0.001222,-6.707280,1.79,0.582216
4,2002-05-01,6.026849,1.796224,0.003142,-5.762878,1.76,0.565314
...,...,...,...,...,...,...,...
282,2025-07-01,333.590679,5.809915,0.055846,-2.885165,4.32,1.463255
283,2025-08-01,340.899950,5.831589,0.253449,-1.372591,4.49,1.501853
284,2025-09-01,348.953859,5.854940,0.216313,-1.531027,4.41,1.483875
285,2025-10-01,366.698338,5.904540,0.076930,-2.564857,4.17,1.427916


In [96]:
# Log Gross Return (Approximation)
# rx_t+1 = (p_t+1 - p_t) + 
monthly_df["r_t+1"] = monthly_df["p_t"].shift(-1) - monthly_df["p_t"] + np.log(1 + monthly_df["D_t"].shift(-1) / monthly_df["P_t"])
monthly_df

,Date,P_t,p_t,D_t,d_t,Rf_t,rf_t,r_t+1
0,2002-01-01,6.747791,1.909215,0.000382,-7.869682,1.69,0.524729,-0.004157
1,2002-02-01,6.713567,1.904130,0.006262,-5.073240,1.69,0.524729,-0.013635
2,2002-03-01,6.619971,1.890091,0.002720,-5.907281,1.78,0.576613,0.015228
3,2002-04-01,6.720307,1.905134,0.001222,-6.707280,1.79,0.582216,-0.108442
4,2002-05-01,6.026849,1.796224,0.003142,-5.762878,1.76,0.565314,-0.044477
...,...,...,...,...,...,...,...,...
282,2025-07-01,333.590679,5.809915,0.055846,-2.885165,4.32,1.463255,0.022434
283,2025-08-01,340.899950,5.831589,0.253449,-1.372591,4.49,1.501853,0.023985
284,2025-09-01,348.953859,5.854940,0.216313,-1.531027,4.41,1.483875,0.049820
285,2025-10-01,366.698338,5.904540,0.076930,-2.564857,4.17,1.427916,0.023259


In [97]:
# Log Excess Return 
monthly_df["rx_t+1"] = monthly_df["r_t+1"] - monthly_df["rf_t"].shift(-1)
monthly_df

,Date,P_t,p_t,D_t,d_t,Rf_t,rf_t,r_t+1,rx_t+1
0,2002-01-01,6.747791,1.909215,0.000382,-7.869682,1.69,0.524729,-0.004157,-0.528886
1,2002-02-01,6.713567,1.904130,0.006262,-5.073240,1.69,0.524729,-0.013635,-0.590248
2,2002-03-01,6.619971,1.890091,0.002720,-5.907281,1.78,0.576613,0.015228,-0.566988
3,2002-04-01,6.720307,1.905134,0.001222,-6.707280,1.79,0.582216,-0.108442,-0.673756
4,2002-05-01,6.026849,1.796224,0.003142,-5.762878,1.76,0.565314,-0.044477,-0.586801
...,...,...,...,...,...,...,...,...,...
282,2025-07-01,333.590679,5.809915,0.055846,-2.885165,4.32,1.463255,0.022434,-1.479419
283,2025-08-01,340.899950,5.831589,0.253449,-1.372591,4.49,1.501853,0.023985,-1.459890
284,2025-09-01,348.953859,5.854940,0.216313,-1.531027,4.41,1.483875,0.049820,-1.378096
285,2025-10-01,366.698338,5.904540,0.076930,-2.564857,4.17,1.427916,0.023259,-1.377924


In [98]:
# Log Excess Return 
monthly_df["Δd_t+1"] = monthly_df["d_t"].shift(-1) - monthly_df["d_t"]
monthly_df

,Date,P_t,p_t,D_t,d_t,Rf_t,rf_t,r_t+1,rx_t+1,Δd_t+1
0,2002-01-01,6.747791,1.909215,0.000382,-7.869682,1.69,0.524729,-0.004157,-0.528886,2.796442
1,2002-02-01,6.713567,1.904130,0.006262,-5.073240,1.69,0.524729,-0.013635,-0.590248,-0.834041
2,2002-03-01,6.619971,1.890091,0.002720,-5.907281,1.78,0.576613,0.015228,-0.566988,-0.799999
3,2002-04-01,6.720307,1.905134,0.001222,-6.707280,1.79,0.582216,-0.108442,-0.673756,0.944402
4,2002-05-01,6.026849,1.796224,0.003142,-5.762878,1.76,0.565314,-0.044477,-0.586801,-0.088809
...,...,...,...,...,...,...,...,...,...,...
282,2025-07-01,333.590679,5.809915,0.055846,-2.885165,4.32,1.463255,0.022434,-1.479419,1.512574
283,2025-08-01,340.899950,5.831589,0.253449,-1.372591,4.49,1.501853,0.023985,-1.459890,-0.158436
284,2025-09-01,348.953859,5.854940,0.216313,-1.531027,4.41,1.483875,0.049820,-1.378096,-1.033831
285,2025-10-01,366.698338,5.904540,0.076930,-2.564857,4.17,1.427916,0.023259,-1.377924,-0.223165


In [99]:
# Log Dividend-Price Ratio
monthly_df["dp_t"] = monthly_df["d_t"] - monthly_df["p_t"]
monthly_df

,Date,P_t,p_t,D_t,d_t,Rf_t,rf_t,r_t+1,rx_t+1,Δd_t+1,dp_t
0,2002-01-01,6.747791,1.909215,0.000382,-7.869682,1.69,0.524729,-0.004157,-0.528886,2.796442,-9.778897
1,2002-02-01,6.713567,1.904130,0.006262,-5.073240,1.69,0.524729,-0.013635,-0.590248,-0.834041,-6.977370
2,2002-03-01,6.619971,1.890091,0.002720,-5.907281,1.78,0.576613,0.015228,-0.566988,-0.799999,-7.797372
3,2002-04-01,6.720307,1.905134,0.001222,-6.707280,1.79,0.582216,-0.108442,-0.673756,0.944402,-8.612414
4,2002-05-01,6.026849,1.796224,0.003142,-5.762878,1.76,0.565314,-0.044477,-0.586801,-0.088809,-7.559102
...,...,...,...,...,...,...,...,...,...,...,...
282,2025-07-01,333.590679,5.809915,0.055846,-2.885165,4.32,1.463255,0.022434,-1.479419,1.512574,-8.695079
283,2025-08-01,340.899950,5.831589,0.253449,-1.372591,4.49,1.501853,0.023985,-1.459890,-0.158436,-7.204180
284,2025-09-01,348.953859,5.854940,0.216313,-1.531027,4.41,1.483875,0.049820,-1.378096,-1.033831,-7.385967
285,2025-10-01,366.698338,5.904540,0.076930,-2.564857,4.17,1.427916,0.023259,-1.377924,-0.223165,-8.469397


##### **2.2) TRAIN/TEST SPLIT**

In [100]:
# Train & Test Split 
monthly_df.to_csv("FVM_data/raw_monthly_df.csv", index=False)

train_monthly_df = monthly_df[monthly_df["Date"] <= "2021-12-31"]
train_monthly_df.to_csv("FVM_data/train_monthly_df.csv", index=False)

test_monthly_df = monthly_df[monthly_df["Date"] >= "2021-12-31"]
test_monthly_df.to_csv("FVM_data/test_monthly_df.csv", index=False)